In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import csv

In [3]:
result = requests.get('http://www.cbr.ru/scripts/XML_daily.asp?date_req=03/09/2023')

In [5]:
with open('cbr.xml', 'w') as f:
  f.write(result.text)

In [ ]:
def export_xml_to_csv_func():
  parser = ET.XMLParser(encoding="UTF-8")
  tree = ET.parse('cbr.xml', parser=parser)
  root = tree.getroot()

  with open('cbr.csv', 'w') as csv_file:
    writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for Valute in root.findall('Valute'):
      NumCode = Valute.find('NumCode').text
      CharCode = Valute.find('CharCode').text
      Nominal = Valute.find('Nominal').text
      Name = Valute.find('Name').text
      Value = Valute.find('Value').text
      writer.writerow([root.attrib['Date']] + [Valute.attrib['ID']] + [NumCode] + [CharCode] + [Nominal] + [Name] + [Value.replace(',', '.')])

export_xml_to_csv_func()

df = pd.read_csv('cbr.csv', header=None)

In [ ]:
rate_df = df.rename(columns={
    0:'rate_dt',
    2:'currency_id',
    3:'currency_cd',
    4:'nominal_qty',
    6:'currency_rate'})[['rate_dt','currency_id','currency_cd', 'nominal_qty', 'currency_rate']]
rate_df

In [ ]:
rate_df

In [123]:
contracts_df = pd.read_excel('/17.5 CASE_CONTRACTS.xlsx', sheet_name = 'Лист1')
contracts_df_fio = pd.read_excel('/17.5 CASE_CLIENTS.xlsx', sheet_name = 'Sheet')
contracts_df_loses = pd.read_excel('/17. 5 CASE_LOSSES.xlsx', sheet_name = 'Лист1')

In [124]:
contracts = pd.merge(contracts_df, contracts_df_fio, on="client_id", how = 'left')

In [125]:
contracts_plus_loses = pd.merge(contracts, contracts_df_loses, on="client_id", how = 'left')

In [127]:
contracts_plus_loses['currency_name'] = contracts_plus_loses['currency_name'].replace('Доллар США', 'No_change')

In [128]:
contracts_plus_loses['currency_name'] = contracts_plus_loses['currency_name'].replace('Российский рубль', 'USD')

In [129]:
contracts_plus_loses = pd.merge(contracts_plus_loses, rate_df, left_on="currency_name", right_on="currency_cd", how = 'left')

In [131]:
contracts_plus_loses['currency_rate'] = contracts_plus_loses['currency_rate'].replace('NaN', 1)
contracts_plus_loses['loss_payout_amt'] = contracts_plus_loses['loss_payout_amt'].replace('NaN', 0)

In [132]:
contracts_plus_loses['loss_payout_amt_USD'] = contracts_plus_loses['loss_payout_amt'] / contracts_plus_loses['currency_rate']

In [133]:
contracts_plus_loses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3711 entries, 0 to 3710
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   contract_id          3711 non-null   int64  
 1   contract_num         3711 non-null   object 
 2   product_name         3711 non-null   object 
 3   client_id            3711 non-null   int64  
 4   contract_status      3711 non-null   object 
 5   currency_name        3711 non-null   object 
 6   duration             3711 non-null   int64  
 7   country              3711 non-null   object 
 8   price                3711 non-null   int64  
 9   insurance_amount     3711 non-null   int64  
 10  last_name            3711 non-null   object 
 11  first_name           3693 non-null   object 
 12  middle_name          3453 non-null   object 
 13  age                  3711 non-null   int64  
 14  sex                  3711 non-null   object 
 15  loss_id              45 non-null     f

In [ ]:
contracts_plus_loses.drop(['rate_dt', 'currency_id', 'currency_cd', 'nominal_qty', 'loss_payout_amt', 'currency_rate'], axis=1, inplace=True)

In [135]:
contracts_plus_loses.to_excel('contracts_plus_loses_dash_0409.xlsx')